# 1

Totalt antal intensivvårdspatienter och avlidna relativt insjuknade per län  

Svar:
- Av de insjuknade intensivvårdades mellan 0.20% (Västermanland) och 0.66% (Sörmland)
- Av de insjunknade avled mellan 0.48% (Halland) och 1.2% (Västermanland)

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px

df = pd.read_excel("../Lab/Data/covid19.xlsx", sheet_name="Totalt antal per region")
df.head()

,Region,Totalt_antal_fall,Fall_per_100000_inv,Totalt_antal_intensivvårdade,Totalt_antal_avlidna
0,Blekinge,30829,19371.449951,85,184
1,Dalarna,75091,26098.780273,260,544
2,Gotland,11874,19776.671875,46,82
3,Gävleborg,74803,26020.503418,332,754
4,Halland,108822,32349.047119,229,518


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Region                        21 non-null     object 
 1   Totalt_antal_fall             21 non-null     int64  
 2   Fall_per_100000_inv           21 non-null     float64
 3   Totalt_antal_intensivvårdade  21 non-null     int64  
 4   Totalt_antal_avlidna          21 non-null     int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 968.0+ bytes


In [52]:
# adding columns showing number of intensive care patients and dead indiviuals relative number of cases

df["Relativt antal intensivvårdade (%)"] = 100*df["Totalt_antal_intensivvårdade"] / df["Totalt_antal_fall"]
df["Relativt antal avlidna (%)"] = 100*df["Totalt_antal_avlidna"] / df["Totalt_antal_fall"]

# sorting on Relativt antal intensivvårdade (%)
df.sort_values(by="Relativt antal intensivvårdade (%)", inplace=True)

df.head()

,Region,Totalt_antal_fall,Fall_per_100000_inv,Totalt_antal_intensivvårdade,Totalt_antal_avlidna,Relativt antal intensivvårdade (%),Relativt antal avlidna (%)
17,Västmanland,78438,28326.636230,156,534,0.198883,0.680792
4,Halland,108822,32349.047119,229,518,0.210435,0.476007
7,Kalmar,62810,25537.878418,135,385,0.214934,0.612960
10,Skåne,346727,24998.670410,819,2587,0.236209,0.746120
0,Blekinge,30829,19371.449951,85,184,0.275714,0.596841


In [57]:
# Plotting new columns as function of Region
fig = px.bar(
    data_frame=df,
    x="Region",
    y=["Relativt antal intensivvårdade (%)", "Relativt antal avlidna (%)"],
    title="Intensivvårdade och avlidna relativt insjuknade per region",
    labels={"value":"Relativt antal (%)", "Region":""},
    barmode="group",
)

# replacing default labels in hover
# ref https://stackoverflow.com/questions/60158618/plotly-how-to-add-elements-to-hover-data-using-plotly-express-piechart
fig.update_traces(hovertemplate = "Region:%{label}: <br>Relativt antal (%): %{value}")

# removing legend title which by default was "variable"
fig.update_layout(legend={"title_text": "", })
fig.show()

fig.write_html("../Lab/Visualiseringar/plot_3_1.html")


# 2

3 län med högsts antal avlidna relativt intensivvårdade och 3 län med lägst antal avlidna relativt intensivvårdade

Svar:
- Högst antal: Västmanland, Halland, Kalmar
- Lägst antal: Uppsala, Norrbotten, Sörmland

In [76]:
# new column with ration dead/intensive care per region created and DataFrame sorted
df["avlidna vs intensivvårdade"] = df["Totalt_antal_avlidna"]/df["Totalt_antal_intensivvårdade"]
df.sort_values(by="avlidna vs intensivvårdade")
highest_ratio = df.head(3)["Region"].tolist()
lowest_ratio = df.tail(3)["Region"].tolist()
print(f"{highest_ratio=}, {lowest_ratio=}")

highest_ratio=['Västmanland', 'Halland', 'Kalmar'], lowest_ratio=['Uppsala', 'Norrbotten', 'Sörmland']


# 3

Vaccination över tid i riket uppdelat på dos

Strategi: 
- läsa in alla DataFrames
- separera minst dos 1 och minst dos 2 i olika DataFrames, resterande är redan separerade
- plocka ut endast rader för Sverige från varje DataFrame
- slå ihop tidskolumnerna 
- lägga ihop data för dos 1, dos2, dos 3, dos 4, dos 5 i samma DataFrame
- göra om tidsräkningen där v52 2020 sätts som 0
- plotta

In [227]:
import pandas as pd

# readin in relevant sheets into separate DataFrames
path = "../Lab/Data/vaccin.xlsx"
df_1_2 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie")
df_3 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie dos 3")
df_4 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie dos 4")
df_5 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie dos 5")
df_1_2.head()

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
0,52,2020,| Sverige |,1639,0.000180,Minst 1 dos
1,52,2020,| Sverige |,0,0.000000,Minst 2 doser
2,52,2020,Stockholm,201,0.000096,Minst 1 dos
3,52,2020,Stockholm,0,0.000000,Minst 2 doser
4,52,2020,Uppsala,5,0.000015,Minst 1 dos


In [228]:
# devide df_1_2 into separate dataframes for Minst 1 dos och Minst 2 doser
df_1 = pd.DataFrame(df_1_2[df_1_2["Vaccinationsstatus"].str.contains("1")]) #??? måste jag göra om til DF manuellt????
df_2 = pd.DataFrame(df_1_2[df_1_2["Vaccinationsstatus"].str.contains("2")])
df_2.head()

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
1,52,2020,| Sverige |,0,0.0,Minst 2 doser
3,52,2020,Stockholm,0,0.0,Minst 2 doser
5,52,2020,Uppsala,0,0.0,Minst 2 doser
7,52,2020,Södermanland,0,0.0,Minst 2 doser
9,52,2020,Östergötland,0,0.0,Minst 2 doser


In [229]:
#?????????????????
# from here a clean function or loop should be used to select rows and change name for column Antal vaccinerade

#df_list = [df_1, df_2, df_3, df_4, df_5]
# for df in df_list:
    # clean function

In [230]:
# select only rows containing Sverige in column Region
df_1 = df_1[df_1["Region"].str.contains("Sverige")]
df_2 = df_2[df_2["Region"].str.contains("Sverige")]
df_3 = df_3[df_3["Region"].str.contains("Sverige")]
df_4 = df_4[df_4["Region"].str.contains("Sverige")]
df_5 = df_5[df_5["Region"].str.contains("Sverige")]

df_2.head()

#????????? should be included in clean function????????????

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
1,52,2020,| Sverige |,0,0.000000,Minst 2 doser
45,53,2020,| Sverige |,0,0.000000,Minst 2 doser
89,1,2021,| Sverige |,0,0.000000,Minst 2 doser
133,2,2021,| Sverige |,68,0.000007,Minst 2 doser
177,3,2021,| Sverige |,7995,0.000879,Minst 2 doser


In [235]:
# combinde cols Vecka and År
df_1["År_v"] = df_1["År"].astype(str) + "v" + df_1["Vecka"].astype(str)
df_2["År_v"] = df_2["År"].astype(str) + "v" + df_2["Vecka"].astype(str)
df_3["År_v"] = df_3["År"].astype(str) + "v" + df_3["Vecka"].astype(str)
df_4["År_v"] = df_4["År"].astype(str) + "v" + df_4["Vecka"].astype(str)
df_5["År_v"] = df_5["År"].astype(str) + "v" + df_5["Vecka"].astype(str)
df_5.head()

#????????? should be included in clean function????????????

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus,År_v
0,33,2022,| Sverige |,1158,0.000529,5 doser,2022v33
22,34,2022,| Sverige |,7569,0.003457,5 doser,2022v34
44,35,2022,| Sverige |,114683,0.052382,5 doser,2022v35
66,36,2022,| Sverige |,327897,0.149768,5 doser,2022v36
88,37,2022,| Sverige |,521400,0.238152,5 doser,2022v37


In [236]:
# set År_v col as index to prepare concatination
df_1 = df_1.set_index("År_v")
df_2 = df_2.set_index("År_v")
df_3 = df_3.set_index("År_v")
df_4 = df_4.set_index("År_v")
df_5 = df_5.set_index("År_v")
df_5.head()


#????????? should be included in clean function????????????

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
År_v,,,,,,
2022v33,33,2022,| Sverige |,1158,0.000529,5 doser
2022v34,34,2022,| Sverige |,7569,0.003457,5 doser
2022v35,35,2022,| Sverige |,114683,0.052382,5 doser
2022v36,36,2022,| Sverige |,327897,0.149768,5 doser
2022v37,37,2022,| Sverige |,521400,0.238152,5 doser


In [237]:
# rename column Antal vaccinerade with values for Vaccinationsstatus
df_1 = df_1.rename(columns={"Antal vaccinerade": df_1["Vaccinationsstatus"][0]})
df_2 = df_2.rename(columns={"Antal vaccinerade": df_2["Vaccinationsstatus"][1]}) # varför måste denna vara [1]???
df_3 = df_3.rename(columns={"Antal vaccinerade": df_3["Vaccinationsstatus"][0]})
df_4 = df_4.rename(columns={"Antal vaccinerade": df_4["Vaccinationsstatus"][0]})
df_5 = df_5.rename(columns={"Antal vaccinerade": df_5["Vaccinationsstatus"][0]})
df_4.head()

#????????? should be included in clean function????????????


,Vecka,År,Region,4 doser,Andel vaccinerade,Vaccinationsstatus
År_v,,,,,,
2022v3,3,2022,| Sverige |,20,0.000002,4 doser
2022v4,4,2022,| Sverige |,1844,0.000221,4 doser
2022v5,5,2022,| Sverige |,6575,0.000788,4 doser
2022v6,6,2022,| Sverige |,12001,0.001438,4 doser
2022v7,7,2022,| Sverige |,23924,0.002866,4 doser


In [270]:
# concatinating all DataFrames into 1 based on col "År_v"

df_1_5 = pd.concat([df_1, df_2, df_3, df_4, df_5], axis = "columns")
df_1_5.columns

Index(['Vecka', 'År', 'Region', 'Minst 1 dos', 'Andel vaccinerade',
       'Vaccinationsstatus', 'Vecka', 'År', 'Region', 'Minst 2 doser',
       'Andel vaccinerade', 'Vaccinationsstatus', 'Vecka', 'År', 'Region',
       '3 doser', 'Andel vaccinerade', 'Vaccinationsstatus', 'Vecka', 'År',
       'Region', '4 doser', 'Andel vaccinerade', 'Vaccinationsstatus', 'Vecka',
       'År', 'Region', '5 doser', 'Andel vaccinerade', 'Vaccinationsstatus'],
      dtype='object')

In [271]:
# selecting relevant columns; replacing NaN with 0
# resetting År_v as column, creating column index corresponding to weeks from start

df_1_5 = df_1_5[['Minst 1 dos', 'Minst 2 doser', '3 doser', '4 doser', '5 doser']].fillna(0).reset_index().reset_index()
df_1_5 = df_1_5.rename(columns={"index": "Veckor från start"})
df_1_5

,Veckor från start,År_v,Minst 1 dos,Minst 2 doser,3 doser,4 doser,5 doser
0,0,2020v52,1639,0,0.0,0.0,0.0
1,1,2020v53,11446,0,0.0,0.0,0.0
2,2,2021v1,37904,0,0.0,0.0,0.0
3,3,2021v2,104762,68,0.0,0.0,0.0
4,4,2021v3,201575,7995,0.0,0.0,0.0
...,...,...,...,...,...,...,...
91,91,2022v38,7806988,7623561,5509988.0,2078006.0,693794.0
92,92,2022v39,7808072,7624734,5516559.0,2191428.0,837547.0
93,93,2022v40,7808932,7625775,5522022.0,2285018.0,942467.0
94,94,2022v41,7809738,7626723,5527028.0,2363358.0,1024426.0


In [273]:
import plotly_express as px

fig = px.line(
    df_1_5,
    x="Veckor från start",
    y=["Minst 1 dos", "Minst 2 doser", "3 doser", "4 doser", "5 doser"],
    title="Antal vaccinerade i Sverige per dos från vaccineringsstart v52 2020",
    labels={"value" : "Ackumulerat antal vaccinerade"}
)
fig.update_layout(legend={'title_text':''})
fig.show()

fig.write_html("../Lab/Visualiseringar/plot_3_3.html")

# 4

Tidsutveckling för total insjuknade i riket jämfört med vaccinerade i riket